In [33]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, nf=32):
        super(UNet, self).__init__()
        
        self.encoder = nn.Sequential(
            UNet._block(in_channels, nf, name="enc1"),
            nn.MaxPool2d(kernel_size=2, stride=2),
            UNet._block(nf, nf * 2, name="enc2"),
            nn.MaxPool2d(kernel_size=2, stride=2),
            UNet._block(nf * 2, nf * 4, name="enc3"),
            nn.MaxPool2d(kernel_size=2, stride=2),
            UNet._block(nf * 4, nf * 8, name="enc4"),        
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.bottleneck = UNet._block(nf * 8, nf * 16, name="bottleneck")
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(nf * 16, nf * 8, kernel_size=2, stride=2),
            UNet._block(nf * 8, nf * 8, name="dec4"),
            nn.ConvTranspose2d(nf * 8, nf * 4, kernel_size=2, stride=2),
            UNet._block(nf * 4, nf * 4, name="dec3"),
            nn.ConvTranspose2d(nf * 4, nf * 2, kernel_size=2, stride=2),
            UNet._block(nf * 2, nf * 2, name="dec2"),
            nn.ConvTranspose2d(nf * 2, nf, kernel_size=2, stride=2),
            UNet._block(nf, nf, name="dec1"),
            nn.Conv2d(in_channels=nf, out_channels=out_channels, kernel_size=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.bottleneck(x)
        x = self.decoder(x)
        return x        
    

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [34]:
unet = UNet()
x = unet.encoder(torch.rand(1,3,256,192))
print(x.shape)
x = unet.bottleneck(x)
print(x.shape)
unet.decoder(x).shape

torch.Size([1, 256, 16, 12])
torch.Size([1, 512, 16, 12])


torch.Size([1, 3, 256, 192])

In [81]:
from collections import OrderedDict

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
from torch.utils.data.dataset import random_split
import matplotlib.pyplot as plt
import torch.optim as optim

In [82]:
device = torch.device("cpu" if not torch.cuda.is_available() else 'cuda')

In [83]:
dataroot = "D:\\TTN-UKM\\FMD\\image\\"
batch_size = 16
image_size = (256, 192) #original image  512, 384

In [85]:
tensor_data = transforms.Compose([
    transforms.Resize(image_size),   #must same as here
    #transforms.CenterCrop(image_size),   -- Take CenterCrop
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) -- 
])



In [86]:
#train_dataset = datasets.ImageFolder(dataroot + 'train', transforms_test)
#test_dataset = datasets.ImageFolder(dataroot + 'test', transforms_test)
#train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
#test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

dataset = datasets.ImageFolder(dataroot, tensor_data)


In [87]:

amt_data = len(dataset)
trainamt = int(amt_data * 0.80)

train_dataset, test_dataset = random_split(dataset, [trainamt, amt_data - trainamt])

In [88]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

In [89]:
from torchvision import datasets, models, transforms
model = models.resnet18() #load resnet18 model
num_features = model.fc.in_features #extract fc layers features


In [90]:
model.fc = nn.Linear(num_features, 10) #(num_of_class == 2)
model = model.to(device)
lossFn = nn.CrossEntropyLoss() #(set loss function)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [91]:
epochs = 200
for epoch in range(epochs):
    for index, (actual, labels) in enumerate(train_dataloader):
        actual = actual.to(device)
        label = labels.to(device)
        optimizer.zero_grad() # optimizer mengosongkan gradient setiap memulai iterasi // deep learning pakai adam optimizer , bukan scd (stochastic gradient decent)
        #pred = unet(actual)
        score = model(actual) # forward gradient
        _, pred = torch.max(score, 1)
        #print(pred, label)
        loss = lossFn(score, label)
        
        loss.backward() # adjust gradient dalam network (gradient ---> weighted)
        optimizer.step()
        
    print("epoch: [{}/{}], loss {:.4f}".format(epoch + 1, epochs, loss.data))

RuntimeError: DataLoader worker (pid(s) 10796) exited unexpectedly